In [2]:
!pip3 install boto3 
!pip3 install sagemaker
!pip3 install tensorflow keras

In [2]:
import boto3
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()  # Use the AWS region configured with the AWS CLI 
#sess = sagemaker.Session(boto3.session.Session(region_name='eu-west-1'))

# This doesn't work on your local machine because it doesn't have an IAM role :)
# role = sagemaker.get_execution_role()

# This is the SageMaker role you're already using, it will work just fine
role = 'arn:aws:iam::776620588530:role/service-role/AmazonSageMaker-ExecutionRole-20210423T144098'

2.38.0


In [3]:
import os
import keras
import numpy as np
from keras.datasets import fashion_mnist

(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
# Train on local data. S3 URIs would work too.
training_input_path   = 'file://data/training.npz'
validation_input_path = 'file://data/validation.npz'

# Store model locally. A S3 URI would work too.
output_path           = 'file:///tmp/model/'

In [5]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='mnist_keras_tf.py',
                          role=role,
                          instance_count=1, 
                          instance_type='local',   # Train on the local CPU ('local_gpu' if it has a GPU)
                          framework_version='1.15', 
                          py_version='py3',
                          hyperparameters={'epochs': 1},
                          output_path=output_path
                         )

In [9]:
# Train! This will pull (once) the SageMaker CPU/GPU container for TensorFlow to your local machine.
# Make sure that Docker is running and that docker-compose is installed

tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

Command output: 


Exception: Failed to run docker,images,-q,763104351884.dkr.ecr.eu-west-2.amazonaws.com/tensorflow-training:1.15-cpu-py3

In [10]:
import docker

!docker images | grep tensorflow

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


In [11]:
!tar tvfz /tmp/model/model.tar.gz

tar (child): /tmp/model/model.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
#1 conda
#2 set region
#3 IAM role
#4 local data
#5 local instance type
#6 docker

